In [1]:
import opengm
import numpy as np
from operator import itemgetter

import sys
import logging
logging.basicConfig(level=logging.DEBUG,
                    format='[%(levelname)s] (%(threadName)-10s) %(message)s',
                    stream=sys.stdout 
)

class FactorGraph(object):
    
    def __init__(self, variables, operator='multiplier'):
        
        assert isinstance(variables, dict)
        
        self.var_names = [vn for vn,_ in variables.items()]
        dimensionality = [d for _,d in variables.items()]
        
        self.gm = opengm.graphicalModel(dimensionality, 
                                        operator=operator)
        
        self.inference = None
        
    
    def add_factor_function(self, variables, probabilities):
        
        if not isinstance(probabilities, np.ndarray):
            probabilities = np.array(probabilities)
        
        if not isinstance(variables, list):
            variables = [variables]
        
        variables = [self.var_names.index(v) for v in variables]
        
        self.gm.addFactor(self.gm.addFunction(probabilities),
                          variables)
        
        self.inference = None
    
    def infer(self):
        self.inference = opengm.inference.BeliefPropagation(self.gm, accumulator='maximizer')
        self.inference.infer()
        
    def get_argmax(self):
        
        if not self.inference:
            self.infer()
            
        argmax = self.inference.arg()
        
        return dict((vn, argmax[i]) for i, vn in enumerate(self.var_names))
    
    def get_marginals(self, marginal_vars):
        
        if not isinstance(marginal_vars, list):
            marginal_vars = [marginal_vars]
        
        if not self.inference:
            self.infer()
        
        marginal_probabilities =  self.inference.marginals(range(len(self.var_names)))
        marginals_ret = {}
        for v in marginal_vars:
            i = self.var_names.index(v)
            marginals_v = marginal_probabilities[i]
            marginals_v /= np.sum(marginals_v)
            marginals_ret[v] = marginals_v
            
        return marginals_ret
    



### Task 2 Solution

In [2]:

m = FactorGraph({'S1':2, 'E1':2})
m.add_factor_function('S1', [0.9, 0.1])           # f(S1)
m.add_factor_function(['S1', 'E1'], [[0, 0.2],    # g(S1,E1)
                                     [0, 0.5]])
m.infer()

argmax = m.get_argmax()

marginal_S1 = m.get_marginals('S1')

print(marginal_S1)
print(argmax)

{'S1': array([ 0.7826087,  0.2173913])}
{'S1': 0, 'E1': 1}


### Task 3

In [3]:
STAGE_MAP = {
    'benign': 1,
    'discovery': 2,
    'access': 3,
    'lateral_movement': 4,
    'privilege_escalation': 5,
    'persistence': 6,
    'defense_evasion': 7,
    'collection': 8,
    'exfiltration': 9,
    'command_control': 10,
    'execution': 11
}

EVENT_MAP = {
    'scan': [1],
    'login': [2],
    'sensitive_uri': [3,4,5],
    'new_kernel_module': [6],
    'dns_tunneling': [7,8,9]
}


ATTACKS = ['benign', 'discovery', 'access', 'lateral_movement', 'privilege_escalation', 'persistence', 'defense_evasion', 'collection', 'exfiltration', 'command_control', 'execution']


ACTIONS = {
    # each value in an actions' vector corresponds to an attack stage
    'NO-OP':   [1.,   0.61, 0.69, 0.09, 0.2 , 0. ,  0.,   0.,   0. ,  0. ,  0.  ],
    'MONITOR': [0.  , 0.39, 0.31 ,0.84, 0.63, 0.7,  0.07 ,0.1 , 0. ,  0. ,  0.  ],
    'STOP':    [0.  , 0.,   0.  , 0.07, 0.17, 0.3,  0.93 ,0.9 , 1. ,  1. ,  1.  ]
}


def get_prob(stages, p, q):
    assert len(p) == len(q) == len(stages)
    prob = np.zeros(11)
    for i in range(len(p)):
        stage_idx = STAGE_MAP[stages[i]] - 1
        prob[stage_idx] = q[i] * (1 - p[i])
    return prob
    
def print_state_belief(mValues):
    mValues = list(mValues)
    print("Beliefs", list(zip(ATTACKS,mValues[0:len(ATTACKS)])))

In [4]:
"""
As an example, we provide the Factor Graph at t=1
Your task is to come up with a general Factor Graph model 
that is parametrized for some general time t
"""
# our sequence of events is simply ['scan']
m = FactorGraph({'E1': 11})
m.add_factor_function('E1', get_prob(['discovery', 'benign'], [0.04, 0.47], [0.5, 0.5]))

m.infer()

argmax = m.get_argmax()
marginal_E1 = m.get_marginals('E1')

print(argmax)
print(marginal_E1)

# argmax(marginal_E1) = 1, which represents the discovery stage (2nd position in array)

# to determine the action to be taken, we look at the probability values
# for the discovery stage for all posible actions, and pick the action
# with the maximum probability
idx = argmax['E1']
action_probabilities = [(k, stage_list[idx]) for k, stage_list in ACTIONS.items()]
print(action_probabilities)
max_action, max_probability = max(action_probabilities, key=itemgetter(1))
print(max_action)

{'E1': 1}
{'E1': array([ 0.3557047,  0.6442953,  0.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       ,  0.       ,  0.       ])}
[('STOP', 0.0), ('MONITOR', 0.39), ('NO-OP', 0.61)]
NO-OP


In [5]:
# HINT: Since you only require the argmax of the ACTION dictionary at each stage,
#       convert the dictionary to a list of actions indexed by stage instead

In [6]:
"""
Build your general model below.
Run your inference for t=1 through t=9
"""
full_sequence = ['scan', 'login', 'sensitive_uri', 'sensitive_uri', 'sensitive_uri',
                 'new_kernel_module', 'dns_tunneling', 'dns_tunneling', 'dns_tunneling']



In [7]:
#We provide the Factor Graph at t=1 for Scan
m = FactorGraph({'S1':11})
m.add_factor_function('S1',get_prob(['discovery','benign'],[0.04,0.47],[0.5,0.5]))
m.infer()

# Calculating marginal probabilities
marginal_S1 = m.get_marginals('S1')
print_state_belief(marginal_S1['S1'])

#Calculating maximum of the probabilities
argmax = m.get_argmax()
print('Attack stage is %s' % ATTACKS[argmax['S1']])

idx = argmax['S1']
action_probabilities = [(k, stage_list[idx]) for k, stage_list in ACTIONS.items()]
print("Action Probabilities", action_probabilities)

max_action, max_probability = max(action_probabilities, key=itemgetter(1))
print("Selected Action is %s" % max_action)

('Beliefs', [('benign', 0.35570469798657717), ('discovery', 0.64429530201342278), ('access', 0.0), ('lateral_movement', 0.0), ('privilege_escalation', 0.0), ('persistence', 0.0), ('defense_evasion', 0.0), ('collection', 0.0), ('exfiltration', 0.0), ('command_control', 0.0), ('execution', 0.0)])
Attack stage is discovery
('Action Probabilities', [('STOP', 0.0), ('MONITOR', 0.39), ('NO-OP', 0.61)])
Selected Action is NO-OP


In [8]:
#We provide the Factor Graph at t = 2 for Login
m = FactorGraph({'S1':11,'S2':11})
m.add_factor_function('S1',get_prob(['discovery','benign'],[0.04,0.47],[0.5,0.5]))
m.add_factor_function('S2',get_prob(['benign'],[0.01],[0.5]))
m.infer()
marginal_S2 = m.get_marginals('S2')
print_state_belief(marginal_S2['S2'])

argmax = m.get_argmax()
print('Attack stage is %s' % ATTACKS[argmax['S2']])

idx = argmax['S2']
action_probabilities = [(k, stage_list[idx]) for k, stage_list in ACTIONS.items()]
print("Action Probabilities", action_probabilities)

max_action, max_probability = max(action_probabilities, key=itemgetter(1))
print("Selected Action is %s" % max_action)

('Beliefs', [('benign', 1.0), ('discovery', 0.0), ('access', 0.0), ('lateral_movement', 0.0), ('privilege_escalation', 0.0), ('persistence', 0.0), ('defense_evasion', 0.0), ('collection', 0.0), ('exfiltration', 0.0), ('command_control', 0.0), ('execution', 0.0)])
Attack stage is benign
('Action Probabilities', [('STOP', 0.0), ('MONITOR', 0.0), ('NO-OP', 1.0)])
Selected Action is NO-OP


In [9]:
#We provide the Factor Graph at t = 3 for  Sensitive URI
m = FactorGraph({'S1':11,'S2':11,'S3':11})
m.add_factor_function('S1',get_prob(['discovery','benign'],[0.04,0.47],[0.5,0.5]))
m.add_factor_function('S2',get_prob(['benign'],[0.01],[0.5]))
m.add_factor_function('S3',get_prob(['privilege_escalation','benign'],[0.02,0.02],[0.1,0.1]))
m.infer()
marginal_S3 = m.get_marginals('S3')
print_state_belief(marginal_S3['S3'])

argmax = m.get_argmax()
print('Attack stage is %s' % ATTACKS[argmax['S3']])

idx = argmax['S3']
action_probabilities = [(k, stage_list[idx]) for k, stage_list in ACTIONS.items()]
print("Action Probabilities", action_probabilities)

max_action, max_probability = max(action_probabilities, key=itemgetter(1))
print("Selected Action is %s" % max_action)

('Beliefs', [('benign', 0.5), ('discovery', 0.0), ('access', 0.0), ('lateral_movement', 0.0), ('privilege_escalation', 0.5), ('persistence', 0.0), ('defense_evasion', 0.0), ('collection', 0.0), ('exfiltration', 0.0), ('command_control', 0.0), ('execution', 0.0)])
Attack stage is benign
('Action Probabilities', [('STOP', 0.0), ('MONITOR', 0.0), ('NO-OP', 1.0)])
Selected Action is NO-OP


In [10]:
#We provide the Factor Graph at t = 4 for  Sensitive URI
m = FactorGraph({'S1':11,'S2':11,'S3':11,'S4':11})
m.add_factor_function('S1',get_prob(['discovery','benign'],[0.04,0.47],[0.5,0.5]))
m.add_factor_function('S2',get_prob(['benign'],[0.01],[0.5]))
m.add_factor_function('S3',get_prob(['privilege_escalation','benign'],[0.02,0.02],[0.1,0.1]))
m.add_factor_function('S4',get_prob(['privilege_escalation','benign'],[0.02,0.02],[0.1,0.1]))
m.infer()
marginal_S4 = m.get_marginals('S4')
print_state_belief(marginal_S4['S4'])

argmax = m.get_argmax()
print('Attack stage is %s' % ATTACKS[argmax['S4']])

idx = argmax['S4']
action_probabilities = [(k, stage_list[idx]) for k, stage_list in ACTIONS.items()]
print("Action Probabilities", action_probabilities)

max_action, max_probability = max(action_probabilities, key=itemgetter(1))
print("Selected Action is %s" % max_action)

('Beliefs', [('benign', 0.5), ('discovery', 0.0), ('access', 0.0), ('lateral_movement', 0.0), ('privilege_escalation', 0.5), ('persistence', 0.0), ('defense_evasion', 0.0), ('collection', 0.0), ('exfiltration', 0.0), ('command_control', 0.0), ('execution', 0.0)])
Attack stage is benign
('Action Probabilities', [('STOP', 0.0), ('MONITOR', 0.0), ('NO-OP', 1.0)])
Selected Action is NO-OP


In [11]:
#We provide the Factor Graph at t = 5 for Sensitive Uri
m = FactorGraph({'S1':11,'S2':11,'S3':11,'S4':11,'S5':11})
m.add_factor_function('S1',get_prob(['discovery','benign'],[0.04,0.47],[0.5,0.5]))
m.add_factor_function('S2',get_prob(['benign'],[0.01],[0.5]))
m.add_factor_function('S3',get_prob(['privilege_escalation','benign'],[0.02,0.02],[0.1,0.1]))
m.add_factor_function('S4',get_prob(['privilege_escalation','benign'],[0.02,0.02],[0.1,0.1]))
m.add_factor_function('S5',get_prob(['privilege_escalation','benign'],[0.02,0.02],[0.1,0.1]))
# Need to add factor function r as privelage_escalation is repeated thrice 
# and according to the table, we need to add corresponding factor function r
m.add_factor_function('S5',get_prob(['privilege_escalation'],[0.05],[0.15]))
m.infer()
marginal_S5 = m.get_marginals('S5')
print_state_belief(marginal_S5['S5'])

argmax = m.get_argmax()
print('Attack stage is %s' % ATTACKS[argmax['S5']])

idx = argmax['S5']
action_probabilities = [(k, stage_list[idx]) for k, stage_list in ACTIONS.items()]
print("Action Probabilities", action_probabilities)

max_action, max_probability = max(action_probabilities, key=itemgetter(1))
print("Selected Action is %s" % max_action)

('Beliefs', [('benign', 0.0), ('discovery', 0.0), ('access', 0.0), ('lateral_movement', 0.0), ('privilege_escalation', 1.0), ('persistence', 0.0), ('defense_evasion', 0.0), ('collection', 0.0), ('exfiltration', 0.0), ('command_control', 0.0), ('execution', 0.0)])
Attack stage is privilege_escalation
('Action Probabilities', [('STOP', 0.17), ('MONITOR', 0.63), ('NO-OP', 0.2)])
Selected Action is MONITOR


In [12]:
# We provide the Factor Graph at t = 6 for New Kernel Module	
m = FactorGraph({'S1':11,'S2':11,'S3':11,'S4':11,'S5':11,'S6':11})
m.add_factor_function('S1',get_prob(['discovery','benign'],[0.04,0.47],[0.5,0.5]))
m.add_factor_function('S2',get_prob(['benign'],[0.01],[0.5]))
m.add_factor_function('S3',get_prob(['privilege_escalation','benign'],[0.02,0.02],[0.1,0.1]))
m.add_factor_function('S4',get_prob(['privilege_escalation','benign'],[0.02,0.02],[0.1,0.1]))
m.add_factor_function('S5',get_prob(['privilege_escalation'],[0.05],[0.15]))
m.add_factor_function('S5',get_prob(['privilege_escalation','benign'],[0.02,0.02],[0.1,0.1]))
# Need to add factor function c as sequence Scan->Sensitive URI->New Kernel Module is repeated thrice 
# and according to the table, we need to add corresponding factor function r
m.add_factor_function('S6',get_prob(['persistence'],[0.006],[0.15]))
m.add_factor_function('S6',get_prob(['persistence','benign'],[0.006,0.0425],[0.05,0.05]))
m.infer()
marginal_S6 = m.get_marginals('S6')
print_state_belief(marginal_S6['S6'])

argmax = m.get_argmax()
print('Attack stage is %s' % ATTACKS[argmax['S6']])

idx = argmax['S6']
action_probabilities = [(k, stage_list[idx]) for k, stage_list in ACTIONS.items()]
print("Action Probabilities", action_probabilities)

max_action, max_probability = max(action_probabilities, key=itemgetter(1))
print("Selected Action is %s" % max_action)

('Beliefs', [('benign', 0.0), ('discovery', 0.0), ('access', 0.0), ('lateral_movement', 0.0), ('privilege_escalation', 0.0), ('persistence', 1.0), ('defense_evasion', 0.0), ('collection', 0.0), ('exfiltration', 0.0), ('command_control', 0.0), ('execution', 0.0)])
Attack stage is persistence
('Action Probabilities', [('STOP', 0.3), ('MONITOR', 0.7), ('NO-OP', 0.0)])
Selected Action is MONITOR


In [13]:
#We provide the Factor Graph at t = 7 for DNS Tunneling
m = FactorGraph({'S1':11,'S2':11,'S3':11,'S4':11,'S5':11,'S6':11,'S7':11})
m.add_factor_function('S1',get_prob(['discovery','benign'],[0.04,0.47],[0.5,0.5]))
m.add_factor_function('S2',get_prob(['benign'],[0.01],[0.5]))
m.add_factor_function('S3',get_prob(['privilege_escalation','benign'],[0.02,0.02],[0.1,0.1]))
m.add_factor_function('S4',get_prob(['privilege_escalation','benign'],[0.02,0.02],[0.1,0.1]))
m.add_factor_function('S5',get_prob(['privilege_escalation'],[0.05],[0.15]))
m.add_factor_function('S5',get_prob(['privilege_escalation','benign'],[0.02,0.02],[0.1,0.1]))
m.add_factor_function('S6',get_prob(['persistence'],[0.006],[0.15]))
m.add_factor_function('S6',get_prob(['persistence','benign'],[0.006,0.0425],[0.05,0.05]))
m.add_factor_function('S7',get_prob(['benign','exfiltration'],[0.05,0.006],[0.15,0.15]))
m.infer()
marginal_S7 = m.get_marginals('S7')
print_state_belief(marginal_S7['S7'])

argmax = m.get_argmax()
print('Attack stage is %s' % ATTACKS[argmax['S7']])

idx = argmax['S7']
action_probabilities = [(k, stage_list[idx]) for k, stage_list in ACTIONS.items()]
print("Action Probabilities", action_probabilities)

max_action, max_probability = max(action_probabilities, key=itemgetter(1))
print("Selected Action is %s" % max_action)

('Beliefs', [('benign', 0.48868312757201648), ('discovery', 0.0), ('access', 0.0), ('lateral_movement', 0.0), ('privilege_escalation', 0.0), ('persistence', 0.0), ('defense_evasion', 0.0), ('collection', 0.0), ('exfiltration', 0.51131687242798352), ('command_control', 0.0), ('execution', 0.0)])
Attack stage is exfiltration
('Action Probabilities', [('STOP', 1.0), ('MONITOR', 0.0), ('NO-OP', 0.0)])
Selected Action is STOP


In [14]:
#We provide the Factor Graph at t = 8 for DNS Tunneling
m = FactorGraph({'S1':11,'S2':11,'S3':11,'S4':11,'S5':11,'S6':11,'S7':11,'S8':11})
m.add_factor_function('S1',get_prob(['discovery','benign'],[0.04,0.47],[0.5,0.5]))
m.add_factor_function('S2',get_prob(['benign'],[0.01],[0.5]))
m.add_factor_function('S3',get_prob(['privilege_escalation','benign'],[0.02,0.02],[0.1,0.1]))
m.add_factor_function('S4',get_prob(['privilege_escalation','benign'],[0.02,0.02],[0.1,0.1]))
m.add_factor_function('S5',get_prob(['privilege_escalation'],[0.05],[0.15]))
m.add_factor_function('S5',get_prob(['privilege_escalation','benign'],[0.02,0.02],[0.1,0.1]))
m.add_factor_function('S6',get_prob(['persistence'],[0.006],[0.15]))
m.add_factor_function('S6',get_prob(['persistence','benign'],[0.006,0.0425],[0.05,0.05]))
m.add_factor_function('S7',get_prob(['benign','exfiltration'],[0.05,0.006],[0.15,0.15]))
m.add_factor_function('S8',get_prob(['benign','exfiltration'],[0.05,0.006],[0.15,0.15]))
m.infer()
marginal_S8 = m.get_marginals('S8')
print_state_belief(marginal_S8['S8'])

argmax = m.get_argmax()
print('Attack stage is %s' % ATTACKS[argmax['S8']])

idx = argmax['S8']
action_probabilities = [(k, stage_list[idx]) for k, stage_list in ACTIONS.items()]
print("Action Probabilities", action_probabilities)

max_action, max_probability = max(action_probabilities, key=itemgetter(1))
print("Selected Action is %s" % max_action)

('Beliefs', [('benign', 0.48868312757201648), ('discovery', 0.0), ('access', 0.0), ('lateral_movement', 0.0), ('privilege_escalation', 0.0), ('persistence', 0.0), ('defense_evasion', 0.0), ('collection', 0.0), ('exfiltration', 0.51131687242798352), ('command_control', 0.0), ('execution', 0.0)])
Attack stage is exfiltration
('Action Probabilities', [('STOP', 1.0), ('MONITOR', 0.0), ('NO-OP', 0.0)])
Selected Action is STOP


In [15]:
#We provide the Factor Graph at t = 9 for DNS Tunneling
m = FactorGraph({'S1':11,'S2':11,'S3':11,'S4':11,'S5':11,'S6':11,'S7':11,'S8':11,'S9':11})
m.add_factor_function('S1',get_prob(['discovery','benign'],[0.04,0.47],[0.5,0.5]))
m.add_factor_function('S2',get_prob(['benign'],[0.01],[0.5]))
m.add_factor_function('S3',get_prob(['privilege_escalation','benign'],[0.02,0.02],[0.1,0.1]))
m.add_factor_function('S4',get_prob(['privilege_escalation','benign'],[0.02,0.02],[0.1,0.1]))
m.add_factor_function('S5',get_prob(['privilege_escalation'],[0.05],[0.15]))
m.add_factor_function('S5',get_prob(['privilege_escalation','benign'],[0.02,0.02],[0.1,0.1]))
m.add_factor_function('S6',get_prob(['persistence'],[0.006],[0.15]))
m.add_factor_function('S6',get_prob(['persistence','benign'],[0.006,0.0425],[0.05,0.05]))
m.add_factor_function('S7',get_prob(['benign','exfiltration'],[0.05,0.006],[0.15,0.15]))
m.add_factor_function('S8',get_prob(['benign','exfiltration'],[0.05,0.006],[0.15,0.15]))
m.add_factor_function('S9',get_prob(['benign','exfiltration'],[0.05,0.006],[0.15,0.15]))
m.infer()
marginal_S9 = m.get_marginals('S9')
print_state_belief(marginal_S9['S9'])

argmax = m.get_argmax()
print('Attack stage is %s' % ATTACKS[argmax['S9']])

idx = argmax['S9']
action_probabilities = [(k, stage_list[idx]) for k, stage_list in ACTIONS.items()]
print("Action Probabilities", action_probabilities)

max_action, max_probability = max(action_probabilities, key=itemgetter(1))
print("Selected Action is %s" % max_action)

('Beliefs', [('benign', 0.48868312757201648), ('discovery', 0.0), ('access', 0.0), ('lateral_movement', 0.0), ('privilege_escalation', 0.0), ('persistence', 0.0), ('defense_evasion', 0.0), ('collection', 0.0), ('exfiltration', 0.51131687242798352), ('command_control', 0.0), ('execution', 0.0)])
Attack stage is exfiltration
('Action Probabilities', [('STOP', 1.0), ('MONITOR', 0.0), ('NO-OP', 0.0)])
Selected Action is STOP
